In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import ccxt
import pandas as pd
import plotly.graph_objs as go

In [2]:
# Set up Binance API
binance = ccxt.binance()
symbol = 'BTC/USDT'

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='live-graph'),
    # html.Div(id='remaining-time'),
    dcc.Interval(
        id='interval-component',
        interval=5 * 1000,  # in milliseconds
        n_intervals=0
    )
])

@app.callback([Output('live-graph', 'figure'), Output('remaining-time', 'children')],
              Input('interval-component', 'n_intervals'))
def update_graph(n):
    ohlcv = binance.fetch_ohlcv(symbol, '1h', limit=1000)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    trace = go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name=symbol
    )
    
    layout = go.Layout(
        title=f'Live OHLCV Chart - {symbol}',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Price'),
        showlegend=True
    )
    
    remaining_time = 10 - (n % 10)  # Calculate remaining seconds
    
    return {'data': [trace], 'layout': layout}, f'Remaining Time until Next Fetch: {remaining_time} seconds'

if __name__ == '__main__':
    app.run_server(debug=False)